**Table of contents**<a id='toc0_'></a>    
- [A step by step approach towards understanding Python Decorators (with a very basic example)](#toc1_)    
- [Real world examples of Python Decorators](#toc2_)    
  - [Example 1: Logging](#toc2_1_)    
  - [Example 2: Function timing](#toc2_2_)    
  - [Example 3: Caching](#toc2_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=5
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# Python Decorators

A decorator is a function that takes another function as an argument and modifies the behavior of the passed function without explicitly changing the source code. Decorators have many different use cases but are typically used when minor changes need to be made to many different functions.

Before we delve into decorators any further, let's first understand two interesting facts about python functions.
- Using Functions as First-Class Citizens: In Python, functions can be assigned to variables, passed as arguments to other functions, and returned from functions. This property makes decorators possible.

- Nested Functions: Python allows functions to be defined within other functions and a decorator is often implemented as a nested function. We will see its benefit shortly.

## <a id='toc1_'></a>[A step by step approach towards understanding Python Decorators (with a very basic example)](#toc0_)

In [1]:
# let's say we have an adder function that adds two numbers
def adder(a, b):
    result = a + b
    print(f"Result: {result}")
    return result

In [2]:
x = adder(5, 3)

Result: 8


In [3]:
print(x)

8


Now, let's say we want to print a welcome message when the function starts to execute and a closing message  when the work is done. Our first instinct would be to use a modified adder function where we use some print satements.

In [4]:
def adder_v2(a, b):
    print("Welcome. The function will now start your calculations.")
    result = a + b
    print(f"Result: {result}")
    print("Success! The function ran without any Errors.")
    return result

In [5]:
x = adder_v2(5, 3)

Welcome. The function will now start your calculations.
Result: 8
Success! The function ran without any Errors.


In [6]:
print(x)

8


But there are some problems with this approach.
1. This becomes very inefficient when we want to add this functionality (of printing welcome messages) to 100s of such other functions. It's not feasible to modify all of them one by one (it's time inefficient and also space inefficient since we have to repeat the same code over and over again). 
2. Even if it's somehow possible (and feasible) to modify all those functions one by one, there remains another problem at hand. It is very likely that the functions we want to modify were written by other people who had very specific tasks in mind when they wrote them. So if we go and change the code of some 100s of functions, it is very likely that we will break the existing code.

So let's think about some solution. One comes to mind. Since we don't want to repeat the same code again and again, can't we define a function to print our message and then just call that function inside the existing functions? Let's try that.

In [7]:
def print_msg():
    print("Welcome. The function will now start your calculations.")
    print("Success! The function ran without any Errors.")

In [8]:
# now let's modify the source code
def adder_v3(a, b):
    print_msg()
    result = a + b
    print(f"Result: {result}")
    return result

In [9]:
x = adder_v3(5, 3)

Welcome. The function will now start your calculations.
Success! The function ran without any Errors.
Result: 8


In [10]:
x

8

As you might've guessed this executes without any error but it's not exactly how we would want it to work. The result is printed at the end and not in between the messages. Even if we ignore that for now, there are still more problems. We would have to call the message printer function inside each of the existing functions if we wanted to modify all those 100s of functions, and this is not very efficient. Also, in this particular case the printer function doesn't take any argument but it will always not be the case and we would've needed to pass the arguments to the modifier function if there were any. So, we are back to square one.

But don't despair. We are in the right path. We just need to be a bit more creative. So what if instead of calling the printer function inside the existing functions we just pass the existing function we want to modify as an argument to the existing function and then call it inside the existing function? Let's see how that works out.

In [11]:
# since functions are first class citizens (more simply they are objects like every other thing in python is),
# we can take them as an argument to other functions, we can return them from inside of a function and also
# we can assign them to variables (that variable can then be used to access the mother function)
def func_modifier(func):
    print("Welcome. The function will now start your calculations.")
    result = func()
    print(f"Result: {result}")
    print("Success! The function ran without any Errors.")
    return result

In [12]:
# now let's see if we've been clever enough
try:
    func_modifier(adder)
except Exception as err_msg:
    print(err_msg)

Welcome. The function will now start your calculations.
adder() missing 2 required positional arguments: 'a' and 'b'


It seems like we've gotten to the 2nd line of the "func_modifier()" function but then it thows an error. This is because the adder function is expecting 2 arguments but we are not passing any arguments to it. So we need to pass the arguments to the adder function as well. So how can we do that?

One approach would be to pass the arguments for the adder function along with the func itself when we are calling the "func_modifier()" function i.e, "func_modifier(adder, a, b)..." (something like that) and then use those "a" and "b" to call the adder function from inside the modifier function. 

But here's the problem with this approach. How do we know that the functions we will be modifying all have the same number of arguments? What if some functions have 2 arguments, some have 3, some have 4 and so on? We can't possibly know that. 

<i> `Enter *args and **kwargs to the rescue.` *args and **kwargs can be used to take in any number of positional and keyword arguments.</i> Let's try it out.

In [13]:
def func_modifier(func, *args, **kwargs):
    print("Welcome. The function will now start your calculations.")
    result = func(*args, **kwargs)
    print("Success! The function ran without any Errors.")
    return result

In [14]:
func_modifier(adder, 5, 3)

Welcome. The function will now start your calculations.
Result: 8
Success! The function ran without any Errors.


8

This is almost exactly what we want but actually we're not quite there yet. Actually we would want to assign an instance of this modified adder function to a variable so that we can use it later on in our code without having to instantiate the function over and over again. Currently we can't do that. This is because the modified adder function is not returning an instance of the function rather it just returns an integer value.

In [15]:
x = func_modifier(adder, 5, 3)

Welcome. The function will now start your calculations.
Result: 8
Success! The function ran without any Errors.


In [16]:
type(x)

int

So let's modify our code so that it returns a function instead of an integer.

In [17]:
def func_modifier_v2(func, *args, **kwargs):
    print("Welcome. The function will now start your calculations.")
    rv = func  # we just want to return the function instead of calling it directly
    print("Success! The function ran without any Errors.")
    return rv

In [18]:
x = func_modifier_v2(adder, 5, 3)

Welcome. The function will now start your calculations.
Success! The function ran without any Errors.


One problem. The print statements are printed prematurely. We don't want them to be printed until the adder function is called. Let's hold it off for a bit and continue, to see if there's any other problem with this code.

In [19]:
x

<function __main__.adder(a, b)>

In [20]:
try:
    x()
except Exception as err_msg:
    print(err_msg)

adder() missing 2 required positional arguments: 'a' and 'b'


Hmm... That's not what we expected. We've already defined, *x = func_modifier_v2(adder, 5, 3)* with the numbers we want to add. So, why it's throwing this error? We thought we fixed it back when we defined with *args and **kwargs i.e, <i>func_modifier(func, *args, **kwargs)</i>. So what's wrong? 

Well, if you look closely, you can see that in our new version, the func_modifier_v2(), in line 3 when we use *rv = func* we don't call it. This is for a reason though. We want to have a function returned and not a number. That's why we wrote it like that. So how we do solve this?

What we can do is, we can call the function instance stored in the variable "x" with arguments "a" and "b".

**Note:** If the code worked then it would've been problematic too. Why? Because we would've had to repeat the *func_modifier(func, a, b)* statement whenever we wanted to use the adder function.

In [21]:
x(3, 5)

Result: 8


8

But this doesn't print our welcome and end messages, which was the main point of creating this modifier function in the first place.

So let's get our priorities straight. In addition to what we've achieved till the "func_modifier(func, *args, **kwargs)" function,
1. We want to print our messages when the function starts to execute and when it's done executing and not before or after that.
2. We want to return a function instance and not a value.

How do we do this? Look no more. *`Nested functions to the rescue`*.

In [22]:
def func_modifier_v3(func, *args, **kwargs):
    def wrapper():
        print("Welcome. The function will now start your calculations.")
        rv = func(*args, **kwargs)
        print("Success! The function ran without any Errors.")
        return rv

    return wrapper

In [23]:
x = func_modifier_v3(adder, 3, 5)

In [24]:
x

<function __main__.func_modifier_v3.<locals>.wrapper()>

In [25]:
x()

Welcome. The function will now start your calculations.
Result: 8
Success! The function ran without any Errors.


8

We're almost there. Think about this a bit more critically. Our purpose of returning a function was to ensure that we can use that function instance later on in our code after assigning it to a variable **once**. But we can't do that with the current code. 

Why? Because we need to supply the "a" and "b" arguments for the adder when we instantiated "x". So it's stuck with the initial values of "a" and "b". If we wanted to use different sets of values we would need to call the "func_modifier_v3()" function again with the new arguments. This completely defeats the puropose of returning a function instance. So what do we do?

We can rewrite our modifier function so that it doesn't directly receive the arguments for the adder function (or whatever function it is called with). Rather, the function it returns i.e, the wrapper() function should receive those *args and **kwargs instead.

In [26]:
def func_modifier_v4(func):
    def wrapper(*args, **kwargs):
        print("Welcome. The function will now start your calculations.")
        rv = func(*args, **kwargs)
        print("Success! The function ran without any Errors.")
        return rv

    return wrapper

In [27]:
x = func_modifier_v4(adder)

In [28]:
x

<function __main__.func_modifier_v4.<locals>.wrapper(*args, **kwargs)>

In [29]:
x(5, 3)

Welcome. The function will now start your calculations.
Result: 8
Success! The function ran without any Errors.


8

In [30]:
x(4, 9)

Welcome. The function will now start your calculations.
Result: 13
Success! The function ran without any Errors.


13

*One last thing. You've probably seen examples of applying decorators with the @ Symbol.* `The common way of applying decorators is using the @ symbol followed by the decorator function name and then defining the source function immediately after that.` This is essentially the same as using, *`func = decorator_func(func)`*.

In [31]:
adder = func_modifier_v4(adder)

In [32]:
adder

<function __main__.func_modifier_v4.<locals>.wrapper(*args, **kwargs)>

In [33]:
adder(5, 5)

Welcome. The function will now start your calculations.
Result: 10
Success! The function ran without any Errors.


10

In [34]:
# the common way of using decorators
@func_modifier_v4
def adder_v3(a, b):
    print_msg()
    result = a + b
    print(f"Result: {result}")
    return result

In [35]:
adder(5, 5)

Welcome. The function will now start your calculations.
Result: 10
Success! The function ran without any Errors.


10

#### <a id='toc1_1_1_'></a>[This in essence, is your Python Decorator!!!](#toc0_)

> A bit more details about the advantages of using the outer function around the wrapper function instead of just a standalone decorator function - 

1. Cleaner and More Readable Code

2. Flexibility with Arguments: In the outer function approach, the decorator (outer function) can accept arguments that can be used to customize the behavior of the decorator (i.e, the wrapper). This allows you to configure the decorator differently for different use cases.

3. Support for Function Signature: The outer function approach maintains the original function's signature. In the standalone wrapper approach, you lose the original function's signature, making it less flexible for functions with varying arguments.

4. Decorator Chaining: With the outer function approach, you can easily chain multiple decorators together by applying them sequentially. This is more challenging with the standalone wrapper approach.

## <a id='toc2_'></a>[Real world examples of Python Decorators](#toc0_)

Decorators are used in various scenarios, including:

- Logging: Record function calls, arguments, and return values for debugging.

- Authentication: Verify user credentials before allowing access to certain functions or routes in web applications.

- Caching: Store and reuse the results of expensive function calls to improve performance.

- Timing: Measure the execution time of functions for performance optimization etc.

### <a id='toc2_1_'></a>[Example 1: Logging](#toc0_)

In [2]:
def log_function_call(func):
    def wrapper(*args, **kwargs):
        print(
            f"The function named '{func.__name__}' was called with args = {args} and kwargs={kwargs}."
        )
        rv = func(*args, **kwargs)
        print(f"Output from called function '{func.__name__}':\n{rv}")
        return rv

    return wrapper

In [3]:
@log_function_call
def add_two(a):
    return a + 2

In [4]:
add_two(10)

The function named 'add_two' was called with args = (10,) and kwargs={}.
Output from called function 'add_two':
12


12

### <a id='toc2_2_'></a>[Example 2: Function timing](#toc0_)

In [12]:
import time

In [21]:
def time_function_execution(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        # time.perf_counter is more suitable for timing function execution than time.time
        rv = func(*args, **kwargs)
        end = time.perf_counter()
        execution_time = end - start
        print(
            f"The function named, '{func.__name__}' took {execution_time} seconds for execution"
        )
        return rv  # comment out to suppress the output from printing when called without assignment

    return wrapper

In [22]:
@time_function_execution
def add_two_to_all_in_range(a):
    lst = []
    for i in range(a):
        lst.append(i + 2)
    return lst

In [23]:
x = add_two_to_all_in_range(10**6)

The function named, 'add_two_to_all_in_range' took 0.12004521099993326 seconds for execution


### <a id='toc2_3_'></a>[Example 3: Caching](#toc0_)

In [36]:
def cacher(func):
    cache_dict = {}

    def wrapper(*args):
        # first check if the value is cached
        if args in cache_dict:
            print("Cache hit!")
            result = cache_dict[args]
        else:
            result = func(*args)
            cache_dict[args] = result  # dict keys are tuples of the form (a, )

        return result

    return wrapper

In [42]:
@cacher
def factorial(a):
    if a <= 1:
        return 1
    else:
        result = a * factorial(a - 1)
    return result

In [43]:
factorial(10)

3628800

In [44]:
factorial(5)

Cache hit!


120